# **VISUALIZACIÓN DE DATOS**

1. Instalación de seaborn

2. Gráficos de tendencia

3. Gráficos de relación entre variables

4. Gráficos de distribución


<img src="../imgs/seaborn.png" width="500px" height="300px">

### **1. Instalación de Seaborn**

In [ ]:
%pip install seaborn

### **2. Gráficos de tendencia**

**Descripción:**

Los gráficos de tendencia se utilizan para representar cómo cambia una variable a lo largo del tiempo o en función de un orden secuencial.

La variable independiente suele ser el tiempo (días, meses, años) u otra métrica ordenada.
La variable dependiente muestra el valor que se analiza (ventas, temperatura, visitas, etc.).

**Ejemplos comunes:**

Gráficos de líneas (line charts).
Gráficos de áreas para resaltar acumulaciones.

**Uso principal:**

Analizar patrones temporales: estacionalidad, crecimiento, decrecimiento.
Identificar picos, tendencias generales o anomalías en los datos.
Aplicaciones:

Seguimiento de métricas de negocio (ventas mensuales, visitas diarias).
Análisis de fenómenos naturales (temperatura, lluvias).


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

El dataset que vamos a utilizar es sobre películas y series de Netflix, se encuentra en la carpeta: **Dia-05/datasets/netflix_titles.csv**

In [ ]:
df_netflix = pd.read_csv("datasets/netflix_titles.csv", index_col=0)
pd.set_option('display.max_rows', 5)

df_netflix.head(10)

#### 2.1. Tendencia del número de lanzamientos por año

Podemos analizar cuántos títulos (películas o programas de TV) se lanzaron cada año. Este gráfico muestra tendencias generales en la producción.

In [ ]:
# Parámetros del gráfico de línea:
# data=launches_per_year: DataFrame que contiene los datos a graficar.
# x='release_year': Columna utilizada para el eje X (año de lanzamiento).
# y='count': Columna utilizada para el eje Y (número de lanzamientos por año).


# Filtrar los datos para agrupar por año de lanzamiento
launches_per_year = df_netflix.groupby('release_year').size().reset_index(name='count')
launches_per_year.tail()
# Gráfico de línea
plt.title('Tendencia de lanzamientos por año')
plt.xlabel('Año de lanzamiento')
plt.ylabel('Número de títulos')

sns.lineplot(data=launches_per_year, x='release_year', y='count')


In [ ]:
# Agregar etiquetas solo para cada 5 años
"""
plt.txt

x: La coordenada en el eje x donde se colocará el texto.
y: La coordenada en el eje y donde se colocará el texto.
s: El texto que deseas mostrar.
**kwargs: Argumentos opcionales para personalizar el estilo del texto.
"""

for i in range(len(launches_per_year)):
    if launches_per_year['release_year'][i] % 5 == 0:  # Etiquetar cada 5 años
        plt.text(
            launches_per_year['release_year'][i],   # Coordenada x
            launches_per_year['count'][i] + 1,     # Coordenada y
            f"{launches_per_year['release_year'][i]}", # Texto
            horizontalalignment='center', size=8
        )
sns.lineplot(data=launches_per_year, x='release_year', y='count')

#### 2.2. Tendencia de duración promedio de películas por año
Podemos analizar cómo ha cambiado la duración promedio de las películas a lo largo del tiempo. Para esto, primero necesitamos limpiar la columna duration.

In [ ]:
# Filtrar solo películas y limpiar la columna "duration"
movies = df_netflix[df_netflix['type'] == 'Movie']
movies['duration'] = movies['duration'].str.replace(' min', '').astype(float)

# Calcular la duración promedio por año
avg_duration_per_year = movies.groupby('release_year')['duration'].mean().reset_index()

# Gráfico de línea
plt.title('Duración promedio de películas por año')
plt.xlabel('Año de lanzamiento')
plt.ylabel('Duración promedio (minutos)')
sns.lineplot(data=avg_duration_per_year, x='release_year', y='duration')


#### 2.3. Evolución del número de títulos por país

Podemos analizar cómo ha cambiado la cantidad de títulos producidos por ciertos países en el tiempo.

In [ ]:
# Parámetros del gráfico de línea:
# data=country_trend: DataFrame que contiene los datos a graficar.
# x='release_year': Columna utilizada para el eje X (año de lanzamiento).
# y='count': Columna utilizada para el eje Y (número de títulos lanzados por año).
# hue='country': Diferencia las líneas por país, asignando un color único a cada país.

# Filtrar los países más frecuentes
top_countries = df_netflix['country'].value_counts().head(5).index
country_trend = df_netflix[df_netflix['country'].isin(top_countries)].groupby(['release_year', 'country']).size().reset_index(name='count')

# Gráfico de línea
plt.title('Evolución del número de títulos por país')
plt.xlabel('Año de lanzamiento')
plt.ylabel('Número de títulos')
plt.legend(title='País', bbox_to_anchor=(1.05, 1), loc='upper left')
sns.lineplot(data=country_trend, x='release_year', y='count', hue='country')


### **3. Gráficos de relación entre variables**


**Descripción:**

Estos gráficos exploran cómo dos o más variables están relacionadas, permitiendo identificar correlaciones, dependencias o patrones complejos.

Ideal cuando ambas variables son continuas, pero también aplicable a combinaciones categóricas-continuas.

**Ejemplos comunes:**

Gráficos de dispersión (scatterplots).
Gráficos de burbujas (scatter con tamaño para una tercera variable).
Mapas de calor (relaciones entre muchas variables).

**Uso principal:**

Detectar correlaciones (positivas, negativas, nulas).
Visualizar concentraciones de datos o valores atípicos.

**Aplicaciones:**

Relación entre ingresos y nivel educativo.
Relación entre precio y demanda de un producto.

#### 3.1. Relación entre la duración y género de las películas
Podemos explorar si ciertos géneros están asociados con películas más largas o más cortas.

In [ ]:
# Parámetros del gráfico de mapa de calor:
# heatmap_data: DataFrame o matriz que contiene los datos para representar en el mapa de calor.
# annot=True: Muestra los valores numéricos en cada celda del mapa.
# fmt=".1f": Formato del texto de las celdas; ".1f" significa un decimal (e.g., 75.5).
# cmap='coolwarm': Paleta de colores que va de azul (valores bajos) a rojo (valores altos).
# cbar_kws={'label': 'Duración promedio (minutos)'}: Personalización de la barra de color (cbar), 
#         en este caso agregando una etiqueta descriptiva ("Duración promedio (minutos)").



movies = df_netflix[df_netflix['type'] == 'Movie']
movies['duration'] = movies['duration'].str.replace(' min', '').astype(float)

# Dividir los géneros y tomar el primero listado en "listed_in"
movies['genre'] = movies['listed_in'].str.split(',').str[0]

# Agrupar por género y calcular la duración promedio de las películas
genre_duration = movies.groupby('genre')['duration'].mean().reset_index()

# Crear un DataFrame pivotado para el mapa de calor
heatmap_data = genre_duration.pivot_table(index='genre', values='duration')

# Crear el mapa de calor
plt.title('Relación entre Género y Duración Promedio (Ordenado)')
plt.xlabel('Duración')
plt.ylabel('Género')

sns.heatmap(heatmap_data, annot=True, fmt=".1f", cmap='coolwarm', cbar_kws={'label': 'Duración promedio (minutos)'})


**Interpretación:**

**Duraciones más largas:**

Géneros como Action & Adventure (113.3 minutos), Classic Movies (112.4 minutos), y Dramas (111.8 minutos) tienen la duración promedio más alta.

**Duraciones más cortas:**

Géneros como Movies (45.9 minutos) y Stand-Up Comedy (66.6 minutos) tienen las duraciones más bajas, lo que es esperado para cortometrajes y espectáculos de comedia.

**Tendencia general:**

Los géneros más orientados a historias elaboradas o épicas tienden a ser más largos (Action & Adventure, Dramas, Sci-Fi & Fantasy).
Géneros ligeros o específicos (Stand-Up Comedy, Children & Family Movies) son más cortos.
Conclusión: La duración promedio de las películas varía significativamente según el género, reflejando las diferencias en narrativa, público objetivo y formato.

#### 3.2. Relación entre la duración y el año de lanzamiento de las películas
Podemos analizar si la duración de las películas ha cambiado con el tiempo.

In [ ]:
# Parámetros del gráfico de dispersión:
# data=scatter_data: El DataFrame que contiene los datos para graficar.
# x='release_year': Columna que se utiliza para el eje X (año de lanzamiento).
# y='duration': Columna que se utiliza para el eje Y (duración de las películas en minutos).
# alpha=0.7: Controla la transparencia de los puntos (0=totalmente transparente, 1=opaco).
# hue='duration': Diferencia los puntos por colores según su duración.
# palette='coolwarm': Paleta de colores que va de azul (valores bajos) a rojo (valores altos).
# size='duration': Escala el tamaño de los puntos según su duración.
# sizes=(20, 200): Define el rango de tamaños para los puntos (20=puntos más pequeños, 200=puntos más grandes).


movies = df_netflix[df_netflix['type'] == 'Movie']
movies['duration'] = movies['duration'].str.replace(' min', '').astype(float)
avg_duration_per_year = movies.groupby('release_year')['duration'].mean().reset_index()

scatter_data = movies[['release_year', 'duration']].dropna()

# Crear el gráfico de dispersión
plt.figure(figsize=(10, 6))
sns.scatterplot(
    data=scatter_data, 
    x='release_year', 
    y='duration', 
    alpha=0.7, 
    hue='duration',  # Diferenciar puntos por su duración usando colores
    palette='coolwarm', 
    size='duration',  # Escalar el tamaño según la duración
    sizes=(20, 200)   # Rango de tamaños para los puntos
)

# Personalización del gráfico
plt.title('Relación entre la duración y el año de lanzamiento de las películas', fontsize=14)
plt.xlabel('Año de lanzamiento', fontsize=12)
plt.ylabel('Duración (minutos)', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.5)

# Crear el gráfico de dispersión
scatter_plot = sns.scatterplot(
    data=scatter_data, 
    x='release_year', 
    y='duration', 
    alpha=0.7, 
    hue='duration',  # Diferenciar puntos por su duración usando colores
    palette='coolwarm', 
    size='duration',  # Escalar el tamaño según la duración
    sizes=(20, 200)   # Rango de tamaños para los puntos
)

# Ajustar la leyenda automáticamente (tamaño y color)
scatter_plot.legend(loc='upper left', bbox_to_anchor=(1.05, 1), title='Duración')

**Interpretación de patrones:**

**Películas más antiguas (antes de 1960):**

Los puntos están más dispersos y, en general, las películas tienden a ser más cortas (mayoría en azul o azul claro).
Esto refleja que en las décadas pasadas, las películas más cortas eran más comunes.

**Películas modernas (después de 2000):**

Hay una mayor concentración de películas entre 80 y 120 minutos, reflejando una tendencia hacia duraciones más estándar en los últimos años.
Algunas películas modernas (punto rojo grande, por ejemplo) son outliers con duraciones excepcionalmente largas.

**Outliers:**

El punto rojo grande en la parte superior derecha (duración >300 minutos) es un outlier. Es una película moderna extremadamente larga que no sigue la tendencia típica.

**Tendencia general:**

Con el tiempo, las películas parecen haber adoptado una duración promedio más estándar (80-120 minutos), con menos películas extremadamente cortas o largas.

### **4. Gráficos de distribución**


**Descripción:**

Estos gráficos muestran cómo los valores de una variable se distribuyen dentro de un rango.

Ayudan a entender la forma, dispersión y centralización de los datos.

**Ejemplos comunes:**

Histogramas.
Gráficos de densidad (KDE plots).
Boxplots (gráficos de caja) para visualizar cuartiles y valores atípicos.

**Uso principal:**

Identificar patrones de distribución: normal, sesgada, multimodal.
Detectar valores atípicos y rangos más comunes.

**Aplicaciones:**

Distribución de edades en una población.
Análisis de ingresos de empleados en una empresa.

#### 4.1. Distribución del número de temporadas de programas de TV
Para los programas de TV, podemos analizar cómo se distribuyen el número de temporadas.

In [ ]:
# Parámetros del gráfico de histograma:
# data=tv_shows: DataFrame que contiene los datos para graficar (solo programas de TV).
# x='seasons': Columna utilizada para el eje X (número de temporadas de los programas de TV).
# bins=15: Divide el rango de datos en 15 intervalos (bins) para el histograma.
# kde=False: No muestra la curva de densidad, solo las barras del histograma.
# color='purple': Especifica el color de las barras del histograma.

# Filtrar únicamente los programas de TV
tv_shows = df_netflix[df_netflix['type'] == 'TV Show']

# Limpiar y transformar la columna 'duration' para obtener el número de temporadas
tv_shows['seasons'] = tv_shows['duration'].str.replace(' Season', '').str.replace('s', '').astype(int)


# Personalización del gráfico
plt.title('Distribución del número de temporadas de programas de TV', fontsize=14)
plt.xlabel('Número de temporadas', fontsize=12)
plt.ylabel('Frecuencia', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)
sns.histplot(data=tv_shows, x='seasons', bins=15, kde=False, color='purple')
